In [1]:
%pip install opencv-python pytube
%pip install yt-dlp opencv-python pandas --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import cv2
import yt_dlp
import zipfile
from pathlib import Path

try:
    # Set up paths
    current_dir = Path(os.getcwd())
    video_path = current_dir / "Tennis.mp4"
    output_folder = current_dir / "frames"
    output_folder.mkdir(exist_ok=True)
    
    # STEP 1: Download video using yt-dlp
    url = "https://www.youtube.com/watch?v=fhn2ANDE4kA"
    
    ydl_opts = { #this is the wanted info for the video/frame dataset that we want to have in
        'format': 'best[ext=mp4]', #format of the video, in which we wanted it to have the best and in mp4 file
        'outtmpl': str(video_path), #output of the video
        'quiet': True
    }
    
    print("Downloading video...")
    with yt_dlp.YoutubeDL(ydl_opts) as ydl: #this is to download the video from the url that is given in youtube. This is an internet one, we could in fact do it ourself if have the time to record it :))
        ydl.download([url])
    
    if not video_path.exists(): #to check if download is okay or not
        raise FileNotFoundError("Video download failed. 'Tennis.mp4' not found.")
    
    print("Video downloaded successfully.")
    #video has been in the file in this step, now, we are using the video for extraction
    
    # STEP 2: Extract frames using OpenCV
    capture = cv2.VideoCapture(str(video_path)) #we will initiate cv2 with VideoCapture here
    
    if not capture.isOpened(): #check if not open correctly
        raise RuntimeError("Failed to open video file with OpenCV.")
    
    # Get video properties
    total_frames = int(capture.get(cv2.CAP_PROP_FRAME_COUNT)) #this is the total frame from that video by using cv2.CAP_PROP_FRAME_COUNT by get, total_frames will have the total number of frame we have
    #to check the number of frame, print it out is a good idea
    print(f"total frames of the whole video is: {total_frames}")
    fps = capture.get(cv2.CAP_PROP_FPS) #this wll give us the fps of the video
    print(f"the fps of this video is {fps}")
    
    seconds_frame = 0.25 #0.25 second intervals captures, this is able to be changed with the in the similar name .py file
    frame_interval = int(fps * seconds_frame)
    print(f"thus, the interval of the frame will be {frame_interval}")
    
    print(f"Video info: {total_frames} frames, {fps} FPS")
    print(f"Extracting 1 frame every {frame_interval} frame (~{seconds_frame} seconds)")
    
    frame_count = 0
    extracted_count = 0
    
    try:
        while True:
            ret, frame = capture.read()
            if not ret:
                break
            
            # Save every 5th frame to reduce total number of images
            if frame_count % frame_interval == 0:
                frame_filename = output_folder / f"frame_{frame_count:05d}.jpg"
                cv2.imwrite(str(frame_filename), frame)
                extracted_count += 1
                
                # Print progress every 100 frames
                if frame_count % 100 == 0:
                    progress = (frame_count / total_frames) * 100
                    print(f"Progress: {progress:.1f}% ({frame_count}/{total_frames} frames)") #check the process of extraction
            
            frame_count += 1
    
    finally:
        capture.release()
    
    print(f"\nExtracted {extracted_count} frames to '{output_folder}'")
    
    # STEP 3: Create ZIP file
    zip_path = current_dir / "frames.zip"
    print(f"\nCreating zip file at: {zip_path}")
    
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf: #open the created zipfile and then write it with each given image/frame we got from the video
        frame_files = list(output_folder.glob('*.jpg')) #make each frame is a .jpg file for correct extraction datatypes
        total_files = len(frame_files) #total length of the files
        
        for i, file_path in enumerate(frame_files, 1):
            zipf.write(file_path, file_path.name)
            if i % 100 == 0:
                print(f"Zipping progress: {(i/total_files)*100:.1f}%")
    
    print(f"\nProcess completed successfully!")
    print(f"- Video saved as: {video_path}")
    print(f"- Frames saved in: {output_folder}")
    print(f"- Zip file created: {zip_path}")
    print(f"- Total frames extracted: {extracted_count}")

except Exception as e:
    print(f"An error occurred: {str(e)}") #if things is not going correctly then this will be raised
    import pytest
    raise ValueError("An error has occured in one of the download and seperating session")
    import traceback
    traceback.print_exc()

Video downloaded successfully.
total frames of the whole video is: 28868
the fps of this video is 60.0
thus, the interval of the frame will be 15
Video info: 28868 frames, 60.0 FPS
Extracting 1 frame every 15 frame (~0.25 seconds)
Progress: 0.0% (0/28868 frames)
Progress: 1.0% (300/28868 frames)
Progress: 2.1% (600/28868 frames)
Progress: 3.1% (900/28868 frames)
Progress: 4.2% (1200/28868 frames)
Progress: 5.2% (1500/28868 frames)
Progress: 6.2% (1800/28868 frames)
Progress: 7.3% (2100/28868 frames)
Progress: 8.3% (2400/28868 frames)
Progress: 9.4% (2700/28868 frames)
Progress: 10.4% (3000/28868 frames)
Progress: 11.4% (3300/28868 frames)
Progress: 12.5% (3600/28868 frames)
Progress: 13.5% (3900/28868 frames)
Progress: 14.5% (4200/28868 frames)
Progress: 15.6% (4500/28868 frames)
Progress: 16.6% (4800/28868 frames)
Progress: 17.7% (5100/28868 frames)
Progress: 18.7% (5400/28868 frames)
Progress: 19.7% (5700/28868 frames)
Progress: 20.8% (6000/28868 frames)
Progress: 21.8% (6300/28868 f